In [1]:
import string
import numpy as np
import pandas as pd
from time import time 
pd.options.mode.chained_assignment = None  # default='warn'

from collections import Counter, defaultdict

from itertools import chain, product
from nltk import FreqDist


dictionary_df = pd.read_csv("dictionary_df1.csv")
dictionary_df.loc[160413, "word"] = "null"
dictionary_df.loc[154353, "word"] = "nan"
dictionary_df = dictionary_df[dictionary_df.columns[1:]]

In [2]:
keys = list(string.ascii_lowercase)
score = [1,3,3,2,1,4,2,4,3,8,5,5,3,2,1,3,10,1,1,1,2,4,4,8,4,10]
score_dict = dict(zip(keys, score))
def score_func(x):
    return sum(np.vectorize(score_dict.get)(x))

In [3]:
def occurrence_letter(board):
    # Build the occurrence of letters matrix
    keys = list(string.ascii_lowercase)
    values = list(np.zeros(26).astype("int64"))
    letter_occur_dict = dict(zip(keys, values))

    # Count the number of occurrence of each letters
    _ = board.flatten()
    _ = _[_ != "None"]
    _.sort()
    letters = "".join(_)
    occurrence = dict(Counter(char for char in letters))

    d1 = letter_occur_dict.copy()
    d1.update(occurrence)
    return d1

def streamline(d1, board):
    # I. Find letters with i occurrence and remove those words not satisfying the conditions

    # Find the largest number of occurrence of each letter
    max_occurrence = max(d1.values())
    dictionary_mod = dictionary_df.copy()
    d1_series = pd.Series(d1)
    for i in range(max_occurrence+1):
        # 1. Locate the letters with i occurrence
        indices = d1_series[d1_series==i].index
        # 2. Find those words not satisfying the conditions in the dictionary and remove from the board
        dictionary_mod = dictionary_mod[(dictionary_mod[indices]>i).apply(sum, axis=1) == 0]

    # Sort the dictionary by score and remove words with length greater than (#tiles - 3)
    rows, columns = board.shape
    df = dictionary_mod.sort_values(by=["score"], ascending=False)
    df = df[df["word len"] <= (rows * columns - 3)].reset_index().iloc[:, 1:]

    
    def bag_letters(x):
        return [x["word"][i:i + 2] for i in range(len(x["word"]) - 1)]

    df["bag_letters"] = df.apply(bag_letters, axis=1)
    return df[df.columns[-4:]]

In [4]:
directions = np.array([
    (0, 1),  # Right
    (0, -1), # Left
    (1, 0),  # Down
    (-1, 0), # Up
    (-1, -1),# Upper left
    (-1, 1), # Upper right
    (1, -1), # Bottom left
    (1, 1)   # Bottom right
])

def DFS(word, board):
    rows, columns = board.shape
    word_len = len(word)
    letters = list(word)
    first_letter = letters[0]
    rs, cs = np.where(board==first_letter)
    locations = list(zip(rs, cs))
    queue = list(zip(locations, np.zeros(len(locations)).astype("int64")))
    visited_lst = []
    V_p = []
    
    while queue:
        q = queue.pop(0)
        visited_lst.append(q)
        V = [[i for i, j in visited_lst if j==k][-1] for k in range(max(visited_lst, key=lambda x:x[1])[1]+1)]
        location, index = q
        if index==(word_len - 1):
            V_p.append(V)
        else:
            locs = location + directions
            locs_valid = locs[(locs[:, 0] < rows) & (locs[:, 1] < columns) & (locs[:, 0] >= 0) & (locs[:, 1] >= 0)]
            neighbors = board[locs_valid[:, 0], locs_valid[:, 1]]
            locs_avai = np.where(neighbors==letters[index+1])
            locations_ = [tuple(l) for l in list(locs_valid[locs_avai]) if tuple(l) not in V]
            if not locations_:
                continue
            indices = (np.ones(len(locations_)) + index).astype("int64")
            queue = list(zip(locations_, indices))+queue
    if V_p:
        return V_p
    return 0

In [5]:
def tiles_remove(tiles, board):
    index = (np.array([i for i, j in tiles]), np.array([j for i, j in tiles]))
    board_1 = board.copy()
    board_1[index] = "None"
    return board_1

def gravity(board_1):
    # Gravity down
    row_index, col_index = np.where(board_1 == "None")
    indices = list(zip(row_index, col_index))
    for r, c in indices:
        column = board_1[:r + 1, c].copy()
        if r > 0:
            column[1:] = column[:-1]
            column[0] = "None"
        board_1[:r + 1, c] = column
    # Gravity right
    row_index, col_index = np.where(board_1 == "None")
    indices = list(zip(row_index, col_index))
    for r, c in indices:
        row = board_1[r, :c + 1].copy()
        if c > 0:
            row[1:] = row[:-1]
            row[0] = "None"
        board_1[r, :c + 1] = row
    return board_1

In [39]:
def Move(board, df, word_lst=[], score=0):
    '''
    Input: board and streamlined dictionary
    
    Aims: 
    1. Find the bag of letters in the board
    2. Further streamlined the dictionary
    3. Check if those words are really in the dictionary
    4. Remove corresponding tiles
    5. Apply gravity to the resulted board
    6. Return (resulted board, word, score)
    '''
    # 1 Find the pairs of letters in the board
    rows, columns = board.shape
    # Check the pair of letters in the grid
    bag_letters_2 = []
    for i in range(rows):
        for j in range(columns):
            if i < rows - 1:
                bag_letters_2.append([
                    board[i, j] + board[i + 1, j],
                    board[i + 1, j] + board[i, j]
                ])
                if j < columns - 1:
                    bag_letters_2.append([
                        board[i, j] + board[i, j + 1],
                        board[i, j + 1] + board[i, j],
                        board[i, j] + board[i + 1, j + 1],
                        board[i + 1, j + 1] + board[i, j]
                    ])
                    if j != 0:
                        bag_letters_2.append([
                            board[i, j] + board[i + 1, j - 1],
                            board[i + 1, j - 1] + board[i, j]
                        ])
                else:
                    bag_letters_2.append([
                        board[i, j] + board[i + 1, j - 1],
                        board[i + 1, j - 1] + board[i, j]
                    ])
            else:
                if j < columns - 1:
                    bag_letters_2.append([
                        board[i, j] + board[i, j + 1],
                        board[i, j + 1] + board[i, j]
                    ])
    bag_letters_2 = list(chain.from_iterable(bag_letters_2))
    bag_letters_2 = {_ for _ in bag_letters_2 if "N" not in list(_)}
    
    # 2. Further streamline the dictionary    
    def bag_letter_avai(x):
        for bag in x["bag_letters"]:
            if bag not in bag_letters_2:
                return False
        return True

    df = df[df.apply(bag_letter_avai, axis=1)]
    
    # 3.  Check if those words are really in the dictionary
    if len(df) == 0:
        return []
    df["tiles"] = df["word"].apply(lambda x: DFS(x, board))
    df = df[df["tiles"]!=0]
    if len(df) == 0:
        return []
    df1 = df.loc[df.index.repeat(df["tiles"].str.len())]
    df1["tiles"] = df["tiles"].explode()
    
    # 4. Remove corresponding tiles
    board1 = df1["tiles"].apply(lambda x: tiles_remove(x, board))
    
    # 5. Apply gravity to the reduced board
    board1 = board1.apply(lambda x: gravity(x))
    
    # 6. Return (resulted board, word, score)
    def Word(x):
        w_lst = [x["word"]]
        w_lst.extend(word_lst)
        return w_lst
    
    output = list(zip(board1, df1.apply(Word, axis=1), df1["score"]+score))
    return output

In [83]:
def DFS_Algorithm(board, step=4):
    '''
    Input: board and streamlined dictionary
    Output: (Best score, combination of words)
    '''
    _ = Move(board, df, [], 0)
    if len(_)==0:
        return []
    queue = list(zip(_, np.ones(len(_)).astype("int64")))
    word_score_lst = [(i[1],i[2]) for i in _]
    max_score = sorted(word_score_lst, key=lambda x: x[1], reverse=True)[0][1]
    best_ = [i for i in _ if i[2]==max_score]
    
    while queue:
        b_tri, index = queue.pop(0)
        b, w, s = b_tri
        strings = b[np.where(b!="None")]
        
        
        if index >= step:
            continue
        if len(strings) == 0:
            continue
        if (s + len(strings)*score_func(strings)) <= max_score:
            continue
        board_lst = Move(b, df, word_lst=w, score=s)
        if index == step-1:
            x = max_score - s
            board_lst = [i for i in board_lst if i[2]>x]
            
        if board_lst:
            _ = list(zip(board_lst, np.ones(len(board_lst))+index))
            bs = sorted(board_lst, key=lambda x: x[2], reverse=True)[0][2]
            if bs > max_score:
                max_score = bs
                print(max_score)
                best_ = [i for i in board_lst if i[2]==max_score]
            queue = _ + queue
        else:
            continue

    return best_

In [93]:
%%time
board = np.array([
    ['z','h','s','s','g','a','s','p','r'],
    ['r','q','q','i','m','n','o','d','e'],
    ['n','b','a','s','t','d','t','i','o'],
    ['e','a','t','k','k','n','a','a','d'],
    ['o','o','r','n','a','y','n','t','o'],
    ['n','p','u','l','r','e','a','o','a'],
    ['t','m','s','e','m','r','p','n','y'],
    ['o','n','e','l','a','s','u','e','a'],
    ['p','h','t','q','m','l','u','r','o'],
    ['i','c','u','o','n','x','o','i','b'],
    ['o','e','n','i','i','r','n','x','k'],
    ['i','q','u','a','p','f','t','l','e']
    ], dtype='<U4')
d1 = occurrence_letter(board)
df = streamline(d1, board)

CPU times: user 9.93 s, sys: 91.3 ms, total: 10 s
Wall time: 10 s


In [89]:
%%time
DFS_Algorithm(board, step=2)

KeyboardInterrupt: 

In [94]:
%%time
word_lst = []
score = 0
while not input():
    l = DFS_Algorithm(board, step=1)
    if len(l) > 0:
        board, word, s = l[0]
        word_lst+=word
        score+=s

        print("*"*50)
        print(board)
        print(word_lst, score)
        print("*"*50)
    else:
        print("No more words")
        print("-"*50)


**************************************************
[['None' 'None' 'None' 'None' 'None' 'z' 'h' 's' 'r']
 ['None' 'None' 'None' 'None' 'r' 'q' 'q' 's' 'e']
 ['None' 'None' 'n' 'b' 'a' 'i' 'g' 'a' 'o']
 ['None' 'e' 'a' 't' 's' 'm' 'd' 'p' 'd']
 ['o' 'o' 'r' 'k' 't' 'n' 'a' 'd' 'o']
 ['n' 'p' 'u' 'n' 'k' 'y' 'a' 'o' 'a']
 ['t' 'm' 's' 'l' 'a' 'r' 'p' 'n' 'y']
 ['o' 'n' 'e' 'e' 'r' 's' 'u' 'e' 'a']
 ['p' 'h' 't' 'q' 'm' 'l' 'u' 'r' 'o']
 ['i' 'c' 'u' 'o' 'n' 'x' 'o' 'i' 'b']
 ['o' 'e' 'n' 'i' 'i' 'r' 'n' 'x' 'k']
 ['i' 'q' 'u' 'a' 'p' 'f' 't' 'l' 'e']]
['lamentations'] 264
**************************************************

**************************************************
[['None' 'None' 'None' 'None' 'None' 'z' 'h' 's' 'r']
 ['None' 'None' 'None' 'None' 'r' 'q' 'q' 's' 'e']
 ['None' 'None' 'None' 'None' 'a' 'i' 'g' 'a' 'o']
 ['None' 'None' 'None' 'None' 's' 'm' 'd' 'p' 'd']
 ['None' 'None' 'o' 'b' 't' 'n' 'a' 'd' 'o']
 ['None' 'n' 'e' 't' 'k' 'y' 'a' 'o' 'a']
 ['t' 'o' 'n' 'k' 'a' 'r'


**************************************************
[['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'None' 'o']
 ['None' 'None' 'None' 'None' 'None' 'None' 'None' 'm' 'q']
 ['None' 'None' 'None' 'None' 'r' 'n' 'g' 'q' 'z']
 ['None' 'None' 'o' 't' 'n' 'x' 'p' 'x' 'r']]
['lamentations', 'quinches', 'quaternary', 'unmanlike', 'anaerobiont', 'slapdash', 'odourful', 'akimbo', 'potpie', 'todays', 'rekes', 'pion'] 1712
**************************************************

N

KeyboardInterrupt: Interrupted by user